In [1]:
# this file collects all results for 8k, under 4 read out modes, up to cls=7 or 10.

In [2]:
from os.path import join, dirname, exists
from os import makedirs
from itertools import chain, islice

from thesis_v2 import dir_dict
from thesis_v2.configs.model.maskcnn_polished_with_rcnn_k_bl import (
    explored_models_20200801_2_generator,
    explored_models_20200801_generator,
    explored_models_20200725_generator,
)
from thesis_v2.analysis.io import collect_rcnn_k_bl_source_analysis

In [3]:
import pandas as pd

In [4]:
f_main_result = join(dir_dict['analyses'], 'yuanyuan_8k_a_3day_refactored', '20200725+20200801+20200801_2', 'source_analysis.pkl')
if not exists(f_main_result):
    makedirs(dirname(f_main_result), exist_ok=True)
    df_main_result = collect_rcnn_k_bl_source_analysis(
        fixed_keys = {
        'kernel_size_l23': 3,
        'kernel_size_l1': 9,
        'ff_1st_block': True,
        'pooling_ksize': 3,
        'pooling_type': 'avg',
        'bn_after_fc': False,
        'rcnn_bl_psize': 1,
        'rcnn_bl_ptype': None,
        'input_size': 50,
        'split_seed': 'legacy',
        'dataset_prefix': 'yuanyuan_8k_a_3day',
        'model_prefix': 'maskcnn_polished_with_rcnn_k_bl',
        'scale_name': '0.01',
        'scale': '0.01',
        'smoothness_name': '0.000005',
        'smoothness': '0.000005',
    },
        generator=chain(
            explored_models_20200801_2_generator(with_source=True),
            explored_models_20200801_generator(with_source=True),
            explored_models_20200725_generator(with_source=True),
        ),
        total_num_param=26,
        train_size_mapping={
            None: 5120,
        },
    )
    df_main_result.to_pickle(f_main_result)
    del df_main_result
df_main_result = pd.read_pickle(f_main_result)

In [5]:
df_main_result

num_param  \
act_fn   ff_1st_bn_before_act loss_type model_seed num_layer out_channel rcnn_bl_cls readout_type train_keep              
relu     False                mse       0          2         8           1           cm-avg       1280            17461   
                                                                                                  2560            17461   
                                                                                                  5120            17461   
                                                                                     cm-last      1280            17461   
                                                                                                  2560            17461   
...                                                                                                                 ...   
softplus True                 poisson   1          6         16          1           legacy       2560            29837   
                                                                                                  5120            29837   
                                                             32          1           legacy       1280            67149   
                                                                                                  2560            67149   
                                                                                                  5120            67149   

                                                                                                                                                source_analysis  
act_fn   ff_1st_bn_before_act loss_type model_seed num_layer out_channel rcnn_bl_cls readout_type train_keep                                                     
relu     False                mse       0          2         8           1           cm-avg       1280                        {('I', 'B1'): 0.7241598808209915}  
                                                                                                  2560                         {('I', 'B1'): 2.134264596867908}  
                                                                                                  5120                         {('I', 'B1'): 2.870888935845869}  
                                                                                     cm-last      1280                        {('I', 'B1'): 0.7241598808209915}  
                                                                                                  2560                         {('I', 'B1'): 2.134264596867908}  
...                                                                                                                                                         ...  
softplus True                 poisson   1          6         16          1           legacy       2560        {('I', 'B1', 'B2', 'B3', 'B4', 'B5'): 0.107387...  
                                                                                                  5120        {('I', 'B1', 'B2', 'B3', 'B4', 'B5'): 0.306178...  
                                                             32          1           legacy       1280        {('I', 'B1', 'B2', 'B3', 'B4', 'B5'): 0.037384...  
                                                                                                  2560        {('I', 'B1', 'B2', 'B3', 'B4', 'B5'): 0.227771...  
                                                                                                  5120        {('I', 'B1', 'B2', 'B3', 'B4', 'B5'): 0.672935...  

[9648 rows x 2 columns]

In [6]:
# compare with previous result.
f_main_result_prev = join(dir_dict['analyses'], 'yuanyuan_8k_a_3day_refactored', '20200725+20200801+20200801_2', 'source_analysis.pkl.legacy-1layer')
df_main_result_prev = pd.read_pickle(f_main_result_prev)

In [7]:
def check_two_same_inner(x_old,x_new):
    if x_old is None and x_new is None:
        return True
    
    assert x_old is not None and x_new is not None
    assert len(x_old) == len(x_new)
    for k_old in x_old.keys():
        k_new = ('I',)+tuple(x+'1' for x in k_old)
        assert x_old[k_old] == x_new[k_new]
    
    return True

def check_two_same(s1, s2):
    assert s1.index.equals(s2.index)
    # check only those legacy ones
    s1 = s1[~((s1.index.get_level_values('num_layer') > 2) | (s1.index.get_level_values('readout_type') =='legacy'))]
    s2 = s2[~((s2.index.get_level_values('num_layer') > 2) | (s2.index.get_level_values('readout_type') =='legacy'))]
    assert s1.index.equals(s2.index)
    print(s1.shape)
    for (x, y) in zip(s1, s2):
        assert check_two_same_inner(x, y)

In [8]:
check_two_same(df_main_result_prev['source_analysis'], df_main_result['source_analysis'])

(4464,)
